<a href="https://colab.research.google.com/github/aswinaus/MS-Copilot-Semantic-Index-API/blob/main/Copilot_Retrieval_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json

# Replace with your actual access token
from google.colab import userdata
GRAPH_TOKEN = userdata.get('GRAPH_TOKEN')
access_token = GRAPH_TOKEN

url = "https://graph.microsoft.com/beta/copilot/retrieval"

headers = {
  "Authorization": f"Bearer {access_token}",
  "Content-Type": "application/json"
}

request_body = {
  "queryString": "Please get me information about how to request for access to EYI MyDocs workspace",
  "dataSource": "sharePoint",
  "resourceMetadata": [
    "title",
    "author"
  ],
  "maximumNumberOfResults": "10"
}

response = requests.post(url, headers=headers, data=json.dumps(request_body))

if response.status_code == 200:
  data = response.json()
  print("API Call Successful:")
  print(json.dumps(data, indent=2))
else:
  print(f"API Call Failed with status code: {response.status_code}")
  print(response.text)

In [ ]:
if response.status_code == 200:
  data = response.json()
  print("API Call Successful:")

  # Rerank the results based on relevance score
  if "retrievalHits" in data:
    reranked_hits = sorted(data["retrievalHits"], key=lambda x: x.get("relevanceScore", 0), reverse=True)
    data["retrievalHits"] = reranked_hits
    print("Results reranked by relevance score.")

  print(json.dumps(data, indent=2))
else:
  print(f"API Call Failed with status code: {response.status_code}")
  print(response.text)

In [ ]:
!pip install autoawq
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
from typing import Tuple, Optional, Union, Dict, Any
from transformers import PreTrainedModel, AutoModel, AutoTokenizer, AutoConfig
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive' # Input a data dir path from your mounted Google Drive
quant_path = f"/{data_dir}/LLMs/Mistral/Mistral-Small-24B-Instruct-2501"

In [ ]:
local_model_path = quant_path
local_tokenizer = AutoTokenizer.from_pretrained(quant_path)
local_model = AutoModel.from_pretrained(quant_path,low_cpu_mem_usage=True)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
local_model.to(device)